In [1]:
%pylab inline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  #交叉验证
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import graphviz
import pickle
import warnings

warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
disorder_R_H = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/avg_R-H_all_dsiorder.pkl','rb'))

data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')
input_lable = np.array(data1.Tier)

In [3]:
all_21_IDR_disorder = []
for position in range(21):
    one_IDR = []
    R_IDR = [i[position] for i in disorder_R_H[0]]
    H_IDR = [i[position] for i in disorder_R_H[1]]
 
    # 转化为二维数组
    for i in range(72):
        test =[]
        test.append(R_IDR[i])
        one_IDR.append(test)
    for i in range(107):
        test =[]
        test.append(H_IDR[i])
        one_IDR.append(test)

    # 所有IDR的平均disorder值添加到一个数组
    all_21_IDR_disorder.append(one_IDR)

In [6]:
disorder_R_H[0][65][15]

0.03288253745995458

In [10]:
# 重新跑关于IDR16由数据更改的情况

# 评估结果
all_21_IDR_length_evaluate = []
# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

one_evaluate = []
for r in range(0,31):
    one_random_state = []
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR_disorder[15],input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = tree.DecisionTreeClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 最优的模型参数
    params = model.best_params_ 
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]

    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0


    one_random_state.append(accuracy)
    one_random_state.append(mcc)
    one_random_state.append(sens)
    one_random_state.append(spec)
    one_evaluate.append(one_random_state)

    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("最优的模型参数是：" + str(params))
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")
all_21_IDR_length_evaluate.append(one_evaluate)
print("以上为IDR__" + str(15 + 1) + "时，evaluation")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.5905854796650598
  sens : 53.333333333333336
  spec : 90.47619047619048
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.10835447742506484
  sens : 58.82352941176471
  spec : 52.63157894736842
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.25495097567963926
  sens : 35.0
  spec : 81.25
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.325
  sens : 62.5
  spec : 70.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.40824829046386296
  sens : 63.63636363636363
  spec : 80.0
数据集划分 random_state：5 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.39605901719066977
  

In [9]:
# 评估结果
all_21_IDR_length_evaluate = []
# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# 21个IDR区域分别进行分类
for position in range(21):
    one_evaluate = []
    for r in range(0,31):
        one_random_state = []
        # train_test_split 按比例划分原数据集
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR_disorder[position],input_lable,train_size=0.8,random_state = r)
        # 定义模型
        model = tree.DecisionTreeClassifier(criterion="entropy")
        # 将模型加入超参数网格搜索和交叉验证     
        model = GridSearchCV(model, param_grid=param_dict, cv=5)
        # 训练数据集
        model = model.fit(Xtrain, Ytrain)
        # 最优的模型参数
        params = model.best_params_ 
        # 模型评估
        predict = model.predict(Xtest)
        cm = confusion_matrix(Ytest,predict)
        tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
        
        n = tp + fp + tn + fn
        accuracy = (tp + tn)/n 
        mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
        sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
        spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
        
        
        one_random_state.append(accuracy)
        one_random_state.append(mcc)
        one_random_state.append(sens)
        one_random_state.append(spec)
        one_evaluate.append(one_random_state)
        
        print("数据集划分 random_state：" + str(r) + " 时，")
        print("Evaluate：")
        print("最优的模型参数是：" + str(params))
        print("  accuracy : " + str(accuracy))
        print("  mcc : " + str(mcc))
        print("  sens : " + str(sens))
        print("  spec : " + str(spec)) 
        print("============================")
    all_21_IDR_length_evaluate.append(one_evaluate)
    print("以上为IDR__" + str(position + 1) + "时，evaluation")
    print("***********************************************************************")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.4166666666666667
  mcc : -0.2789511878128255
  sens : 17.647058823529413
  spec : 63.1578947368421
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.4444444444444444
  mcc : -0.08838834764831843
  sens : 10.0
  spec : 87.5
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.5277777777777778
  mcc : -0.0642575463121999
  sens : 12.5
  spec : 85.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'max_depth': 15, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.0032079140624165347
  sens : 36.36363636363637
  spec : 64.0
数据集划分 random_state：5 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.4722222222222222
  mcc : -0.04159451654038515


数据集划分 random_state：8 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.6673806766489837
  sens : 42.857142857142854
  spec : 95.45454545454545
数据集划分 random_state：9 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.39605901719066977
  sens : 47.368421052631575
  spec : 82.35294117647058
数据集划分 random_state：10 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.4034466888812335
  sens : 43.75
  spec : 85.0
数据集划分 random_state：11 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6944444444444444
  mcc : 0.539972451087909
  sens : 50.0
  spec : 88.88888888888889
数据集划分 random_state：12 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.6622661785325219
  sens : 47.05882352941176
  spec : 94.73684210526315
数据集划分 random_state：13 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accur

数据集划分 random_state：15 时，
Evaluate：
最优的模型参数是：{'max_depth': 25, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.07857752591619217
  sens : 41.66666666666667
  spec : 66.66666666666666
数据集划分 random_state：16 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.3965923098747467
  sens : 84.61538461538461
  spec : 65.21739130434783
数据集划分 random_state：17 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.3774256780481986
  sens : 50.0
  spec : 84.61538461538461
数据集划分 random_state：18 时，
Evaluate：
最优的模型参数是：{'max_depth': 15, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.12598815766974242
  sens : 54.54545454545454
  spec : 57.14285714285714
数据集划分 random_state：19 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6944444444444444
  mcc : 0.3743524429635728
  sens : 38.46153846153847
  spec : 86.95652173913044
数据集划分 random_state：20 时，
Evaluate：
最优的模型参数是：{'max_depth': 

数据集划分 random_state：22 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.34966263280340226
  sens : 63.63636363636363
  spec : 76.0
数据集划分 random_state：23 时，
Evaluate：
最优的模型参数是：{'max_depth': 25, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.24192589470360243
  sens : 57.14285714285714
  spec : 68.18181818181817
数据集划分 random_state：24 时，
Evaluate：
最优的模型参数是：{'max_depth': 15, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.24618298195866545
  sens : 35.714285714285715
  spec : 81.81818181818183
数据集划分 random_state：25 时，
Evaluate：
最优的模型参数是：{'max_depth': 15, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.17407765595569785
  sens : 42.857142857142854
  spec : 72.72727272727273
数据集划分 random_state：26 时，
Evaluate：
最优的模型参数是：{'max_depth': 25, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.27386127875258304
  sens : 66.66666666666666
  spec : 66.66666666666666
数据集划分 random_state：27 时，
Evaluate：
最优

数据集划分 random_state：29 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.3424747597107866
  sens : 35.714285714285715
  spec : 86.36363636363636
数据集划分 random_state：30 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.06493506493506493
  sens : 42.857142857142854
  spec : 63.63636363636363
以上为IDR__5时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.4629100498862757
  sens : 20.0
  spec : 95.23809523809523
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.5445100807792042
  sens : 29.411764705882355
  spec : 94.73684210526315
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.7302967433402215
  sens 

...

数据集划分 random_state：27 时，
Evaluate：
最优的模型参数是：{'max_depth': 25, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.1543033499620919
  sens : 50.0
  spec : 71.42857142857143
数据集划分 random_state：28 时，
Evaluate：
最优的模型参数是：{'max_depth': 25, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.28210976821983985
  sens : 61.53846153846154
  spec : 69.56521739130434
数据集划分 random_state：29 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : -0.0778498944161523
  sens : 14.285714285714285
  spec : 81.81818181818183
数据集划分 random_state：30 时，
Evaluate：
最优的模型参数是：{'max_depth': 15, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.5194344725754872
  sens : 57.14285714285714
  spec : 86.36363636363636
以上为IDR__15时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.5905854796650598
  sens : 53.333333333333336
  

数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.5
  mcc : -0.0125
  sens : 43.75
  spec : 55.00000000000001
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.16447301381396023
  sens : 27.27272727272727
  spec : 84.0
数据集划分 random_state：5 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.4444444444444444
  mcc : -0.29554023164452436
  sens : 5.263157894736842
  spec : 88.23529411764706
数据集划分 random_state：6 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : -0.5118906968889915
  sens : 0.0
  spec : 90.9090909090909
数据集划分 random_state：7 时，
Evaluate：
最优的模型参数是：{'max_depth': 50, 'random_state': 0}
  accuracy : 0.4444444444444444
  mcc : 0.009183204950474644
  sens : 62.5
  spec : 39.285714285714285
数据集划分 random_state：8 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6666666666666666


数据集划分 random_state：10 时，
Evaluate：
最优的模型参数是：{'max_depth': 25, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.3265986323710904
  sens : 25.0
  spec : 90.0
数据集划分 random_state：11 时，
Evaluate：
最优的模型参数是：{'max_depth': 30, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.3557562367689427
  sens : 61.111111111111114
  spec : 72.22222222222221
数据集划分 random_state：12 时，
Evaluate：
最优的模型参数是：{'max_depth': 15, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.24654189620569977
  sens : 47.05882352941176
  spec : 73.68421052631578
数据集划分 random_state：13 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.4444444444444444
  mcc : -0.2175970699446223
  sens : 21.428571428571427
  spec : 59.09090909090909
数据集划分 random_state：14 时，
Evaluate：
最优的模型参数是：{'max_depth': 25, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.30929478706587094
  sens : 33.33333333333333
  spec : 85.71428571428571
数据集划分 random_state：15 时，
Evaluate：
最优的模型参数是：{'max_de

数据集划分 random_state：17 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.3076923076923077
  sens : 50.0
  spec : 80.76923076923077
数据集划分 random_state：18 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.5277777777777778
  mcc : 0.11664236870396087
  sens : 45.45454545454545
  spec : 64.28571428571429
数据集划分 random_state：19 时，
Evaluate：
最优的模型参数是：{'max_depth': 30, 'random_state': 0}
  accuracy : 0.3888888888888889
  mcc : -0.26390913801210825
  sens : 23.076923076923077
  spec : 47.82608695652174
数据集划分 random_state：20 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.14920941939059815
  sens : 50.0
  spec : 66.66666666666666
数据集划分 random_state：21 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.5
  mcc : 0.0
  sens : 44.44444444444444
  spec : 55.55555555555556
数据集划分 random_state：22 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  acc

数据集划分 random_state：24 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6944444444444444
  mcc : 0.5667212899899379
  sens : 28.57142857142857
  spec : 95.45454545454545
数据集划分 random_state：25 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.04351941398892446
  sens : 35.714285714285715
  spec : 68.18181818181817
数据集划分 random_state：26 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.19644381479048043
  sens : 50.0
  spec : 70.83333333333334
数据集划分 random_state：27 时，
Evaluate：
最优的模型参数是：{'max_depth': 15, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.03857583749052298
  sens : 37.5
  spec : 67.85714285714286
数据集划分 random_state：28 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : -0.14744195615489714
  sens : 7.6923076923076925
  spec : 86.95652173913044
数据集划分 random_state：29 时，
Evaluate：
最优的模型参数是：{'max_dep

In [13]:
for k in range(21):
    IDR_acc = [i[0] for i in all_21_IDR_length_evaluate[k]]

    print("IDR_" + str(k+1) + "__max_acc_all_evaluate：" + str(all_21_IDR_length_evaluate[k][IDR_acc.index(np.max(IDR_acc))])  
          + "  index_" +  str(IDR_acc.index(np.max(IDR_acc))))

IDR_1__max_acc_all_evaluate：[0.75, 0.4681910919173118, 61.53846153846154, 82.6086956521739]  index_16
IDR_2__max_acc_all_evaluate：[0.7777777777777778, 0.565752381856018, 64.28571428571429, 86.36363636363636]  index_25
IDR_3__max_acc_all_evaluate：[0.75, 0.3774256780481986, 50.0, 84.61538461538461]  index_17
IDR_4__max_acc_all_evaluate：[0.7222222222222222, 0.28644594961577313, 30.0, 88.46153846153845]  index_17
IDR_5__max_acc_all_evaluate：[0.75, 0.33949371345684337, 75.0, 75.0]  index_7
IDR_6__max_acc_all_evaluate：[0.8055555555555556, 0.6285393610547089, 58.333333333333336, 91.66666666666666]  index_20
IDR_7__max_acc_all_evaluate：[0.8055555555555556, 0.6937819061732104, 45.45454545454545, 96.0]  index_4
IDR_8__max_acc_all_evaluate：[0.6388888888888888, 0.18883492020068354, 54.54545454545454, 68.0]  index_4
IDR_9__max_acc_all_evaluate：[0.8333333333333334, 0.7374757716174346, 54.54545454545454, 96.0]  index_4
IDR_10__max_acc_all_evaluate：[0.75, 0.40859257313228114, 30.0, 92.3076923076923]  

In [11]:
for k in range(21):
    IDR_acc = [i[0] for i in all_21_IDR_length_evaluate[k]]
    print("IDR_" + str(k+1) + "__max_acc_all_evaluate：" + str(all_21_IDR_length_evaluate[k][IDR_acc.index(np.max(IDR_acc))]))

IDR_1__max_acc_all_evaluate：[0.75, 0.4681910919173118, 61.53846153846154, 82.6086956521739]
IDR_2__max_acc_all_evaluate：[0.7777777777777778, 0.565752381856018, 64.28571428571429, 86.36363636363636]
IDR_3__max_acc_all_evaluate：[0.75, 0.3774256780481986, 50.0, 84.61538461538461]
IDR_4__max_acc_all_evaluate：[0.7222222222222222, 0.28644594961577313, 30.0, 88.46153846153845]
IDR_5__max_acc_all_evaluate：[0.75, 0.33949371345684337, 75.0, 75.0]
IDR_6__max_acc_all_evaluate：[0.8055555555555556, 0.6285393610547089, 58.333333333333336, 91.66666666666666]
IDR_7__max_acc_all_evaluate：[0.8055555555555556, 0.6937819061732104, 45.45454545454545, 96.0]
IDR_8__max_acc_all_evaluate：[0.6388888888888888, 0.18883492020068354, 54.54545454545454, 68.0]
IDR_9__max_acc_all_evaluate：[0.8333333333333334, 0.7374757716174346, 54.54545454545454, 96.0]
IDR_10__max_acc_all_evaluate：[0.75, 0.40859257313228114, 30.0, 92.3076923076923]
IDR_11__max_acc_all_evaluate：[0.8055555555555556, 0.5345224838248488, 83.33333333333334

In [12]:
# 评估结果
all_IDR_avg_disorder_evaluate = []

# 将21个区域的平均disorder值一起作为特征
all_R_H_IDR_disorder =[]
all_R_H_IDR_disorder.extend(disorder_R_H[0])
all_R_H_IDR_disorder.extend(disorder_R_H[1])

# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    one_random_state = []
    # 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_R_H_IDR_disorder, input_lable, train_size=0.8, random_state=r)
    # 定义模型  
    clf = tree.DecisionTreeClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5) #将训练/测试数据集划分5个互斥子集
    # 训练数据集
    clf = clf.fit(Xtrain, Ytrain)  
    # 最优的模型参数
    params = clf.best_params_ 
    
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred) # 混淆矩阵
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    
    one_random_state.append(accuracy)
    one_random_state.append(mcc)
    one_random_state.append(sens)
    one_random_state.append(spec)  
    all_IDR_avg_disorder_evaluate.append(one_random_state)
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("最优的模型参数是：" + str(params))
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 9}
  accuracy : 0.8333333333333334
  mcc : 0.7035243599527023
  sens : 73.33333333333333
  spec : 90.47619047619048
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 3}
  accuracy : 0.7222222222222222
  mcc : 0.42463241153065945
  sens : 76.47058823529412
  spec : 68.42105263157895
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 0}
  accuracy : 0.6944444444444444
  mcc : 0.5453453906108109
  sens : 55.00000000000001
  spec : 87.5
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'random_state': 24}
  accuracy : 0.7222222222222222
  mcc : 0.4216370213557839
  sens : 75.0
  spec : 70.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 6}
  accuracy : 0.7777777777777778
  mcc : 0.4626219273003039
  sens : 81.81818181818183
  spec : 76.0
数据集划分 random_state：5 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'random_state': 6}
  accuracy 

In [13]:
np.max([i[0] for i in all_IDR_avg_disorder_evaluate])

0.9444444444444444